In [10]:
import openpyxl
import os

def copy_cell_styles(source_cell, target_cell):
    """Copy the style attributes from the source cell to the target cell."""
    target_cell.font = openpyxl.styles.Font(name=source_cell.font.name,
                                            size=source_cell.font.size,
                                            bold=source_cell.font.bold,
                                            italic=source_cell.font.italic,
                                            vertAlign=source_cell.font.vertAlign,
                                            underline=source_cell.font.underline,
                                            strike=source_cell.font.strike,
                                            color=source_cell.font.color)
    
    target_cell.fill = openpyxl.styles.PatternFill(fill_type=source_cell.fill.fill_type,
                                                   start_color=source_cell.fill.start_color,
                                                   end_color=source_cell.fill.end_color)
    
    target_cell.border = openpyxl.styles.Border(left=source_cell.border.left,
                                                right=source_cell.border.right,
                                                top=source_cell.border.top,
                                                bottom=source_cell.border.bottom)
    
    target_cell.alignment = openpyxl.styles.Alignment(horizontal=source_cell.alignment.horizontal,
                                                      vertical=source_cell.alignment.vertical,
                                                      text_rotation=source_cell.alignment.text_rotation,
                                                      wrap_text=source_cell.alignment.wrap_text,
                                                      shrink_to_fit=source_cell.alignment.shrink_to_fit,
                                                      indent=source_cell.alignment.indent)
    
    target_cell.number_format = source_cell.number_format
    target_cell.protection = openpyxl.styles.Protection(locked=source_cell.protection.locked,
                                                        hidden=source_cell.protection.hidden)

def copy_dimensions(source_sheet, target_sheet):
    """Copy row heights and column widths from source sheet to target sheet."""
    for col_letter, col_dim in source_sheet.column_dimensions.items():
        target_sheet.column_dimensions[col_letter].width = col_dim.width
    
    for row_idx, row_dim in source_sheet.row_dimensions.items():
        target_sheet.row_dimensions[row_idx].height = row_dim.height

def copy_merged_cells(source_sheet, target_sheet, row_offset, is_header=False, header_row=None):
    """Copy merged cell ranges from source sheet to target sheet with an offset."""
    for merged_range in source_sheet.merged_cells.ranges:
        min_row = merged_range.min_row
        max_row = merged_range.max_row
        min_col = merged_range.min_col
        max_col = merged_range.max_col

        if is_header:
            # Apply header merged ranges only to the top of the target sheet
            if max_row <= header_row:
                target_sheet.merge_cells(start_row=min_row, start_column=min_col,
                                         end_row=max_row, end_column=max_col)
        else:
            # Apply merged ranges for data, adjusted for the row offset
            if min_row > header_row:
                target_sheet.merge_cells(start_row=min_row + row_offset, start_column=min_col,
                                         end_row=max_row + row_offset, end_column=max_col)





def copy_data_validations(source_sheet, target_sheet, row_offset, header_row):
    """Copy data validation rules from source sheet to target sheet with an offset."""
    for data_validation in source_sheet.data_validations.dataValidation:
        dv_new = openpyxl.worksheet.datavalidation.DataValidation(
            type=data_validation.type,
            formula1=data_validation.formula1,
            formula2=data_validation.formula2,
            allow_blank=data_validation.allowBlank,
            operator=data_validation.operator,
            showErrorMessage=data_validation.showErrorMessage,
            showInputMessage=data_validation.showInputMessage,
            promptTitle=data_validation.promptTitle,
            prompt=data_validation.prompt,
            errorTitle=data_validation.errorTitle,
            error=data_validation.error,
        )
        for cell_range in data_validation.ranges:
            min_row, max_row, min_col, max_col = (
                cell_range.min_row,
                cell_range.max_row,
                cell_range.min_col,
                cell_range.max_col,
            )
            if min_row > header_row:  # Adjust data rows only
                dv_new.add(f"{openpyxl.utils.get_column_letter(min_col)}{min_row + row_offset}:"
                           f"{openpyxl.utils.get_column_letter(max_col)}{max_row + row_offset}")
            else:  # Keep header validations as is
                dv_new.add(f"{openpyxl.utils.get_column_letter(min_col)}{min_row}:"
                           f"{openpyxl.utils.get_column_letter(max_col)}{max_row}")
        target_sheet.add_data_validation(dv_new)

def merge_excel(input_folder, output_file, header_row):
    try:
        # Get all Excel files in the input folder
        excel_files = [f for f in os.listdir(input_folder) if f.endswith('.xlsx')]

        # Create a new workbook for the merged output
        wb_merged = openpyxl.Workbook()
        sheet_merged = wb_merged.active
        sheet_merged.title = "MergedData"

        row_offset = 0  # Keep track of the row offset for each file

        for i, file in enumerate(excel_files):
            filepath = os.path.join(input_folder, file)
            wb = openpyxl.load_workbook(filepath)
            sheet = wb.active

            # Copy dimensions and merged cells for the first file
            if i == 0:
                copy_dimensions(sheet, sheet_merged)

            # Copy merged cells for the header part
            if i == 0:
                copy_merged_cells(sheet, sheet_merged, row_offset, is_header=True, header_row=header_row)

            # Copy rows
            for row_idx, row in enumerate(sheet.iter_rows(), start=1):
                # 跳过非首文件的表头行
                if row_idx <= header_row and i > 0:
                    continue  
            
                # 跳过完全空的行
                if all(cell.value is None for cell in row):
                    continue
            
                target_row = row_idx + row_offset
                for col_idx, source_cell in enumerate(row, start=1):
                    target_cell = sheet_merged.cell(row=target_row, column=col_idx, value=source_cell.value)
                    copy_cell_styles(source_cell, target_cell)



            
            # Copy merged cells for the data part
            copy_merged_cells(sheet, sheet_merged, row_offset, is_header=False, header_row=header_row)

            # Copy data validations
            copy_data_validations(sheet, sheet_merged, row_offset, header_row)

            # Update the row offset

        
            # row_offset += sheet.max_row - (header_row if i > 0 else 0)
          

            # 获取有效行数
  
            max_row = max(
                (cell.row for row in sheet.iter_rows(min_row=header_row + 1) for cell in row if cell.value is not None),
                default=header_row
            )
        
            row_offset += max_row - (header_row if i > 0 else 0)
            # row_offset += sheet.max_row - header_row if i > 0 else sheet.max_row
     
        # Save the merged workbook
        wb_merged.save(output_file)
        print(f"Merged file saved: {output_file}")

    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
input_folder = r'/Users/gejundeng/Desktop/拆分文件夹'  # Folder containing split files
output_file = r'/Users/gejundeng/Desktop/合并结果.xlsx'  # Path for the merged file
header_row = 4  # Header row (1-based index)

merge_excel(input_folder, output_file, header_row)

Merged file saved: /Users/gejundeng/Desktop/合并结果.xlsx
